In [1]:
import pandas as pd 
import numpy as np
import os
import pyspark 
from pyspark.sql import SparkSession

In [2]:
#replace with correct directory where data files are located 
os.chdir(r'C:\Users\Nicho\gc-march-2022\data')

In [3]:
da = pd.read_csv('Indeed_DataAnalyst_2022.csv')
de = pd.read_csv('Indeed_DataEngineer_2022.csv')
ds = pd.read_csv('Indeed_DataScience_2022.csv')

indeed_historic = pd.read_csv('indeed_jobs.csv')
indeed_historic['datadate'] = 'Fall 2021'


In [4]:
da.info()
de.info()
ds.info()
indeed_historic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1155 non-null   int64 
 1   title        1155 non-null   object
 2   company      1153 non-null   object
 3   salary       388 non-null    object
 4   description  1153 non-null   object
 5   location     1153 non-null   object
dtypes: int64(1), object(5)
memory usage: 54.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1155 non-null   int64 
 1   title        1155 non-null   object
 2   company      1155 non-null   object
 3   salary       398 non-null    object
 4   description  1155 non-null   object
 5   location     1155 non-null   object
dtypes: int64(1), object(5)
memory usage: 54.3+ KB
<class 'pandas.core.frame.Data

There is a lot more rows in the engineering data than the other two sites. Only the salary column contains null values.

In [5]:
indeed_df = pd.concat([da,de,ds])
indeed_df['datadate'] = 'Spring 2022'
indeed_df = pd.concat([indeed_df,indeed_historic])
indeed_df = indeed_df[['title','company','salary','description','location','datadate']]
indeed_df.drop_duplicates(inplace=True)
indeed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3262 entries, 0 to 2269
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        3262 non-null   object
 1   company      2346 non-null   object
 2   salary       842 non-null    object
 3   description  3254 non-null   object
 4   location     3260 non-null   object
 5   datadate     3262 non-null   object
dtypes: object(6)
memory usage: 178.4+ KB


In [6]:
indeed_df['salary'] = indeed_df['salary'].str.lower()
indeed_df['description'] = indeed_df['description'].str.lower()
indeed_df['location'] = indeed_df['location'].str.lower()

In [7]:
indeed_df.head(5)

,title,company,salary,description,location,datadate
0,Data Science Analyst- Marketing,Indeed,"$100,000 - $140,000 a year",work across a wide range of data engineering t...,remote in texas+1 location,Spring 2022
1,Data Analyst/Statistician,US Department of Veterans Affairs,"$51,980 - $103,518 a year",create processes for assessing quality and acc...,"hybrid remote in ann arbor, mi 48105",Spring 2022
2,Departmental Analyst 9-P11 - Data Analytics,State of Michigan,NaN,"and reporting, including biannual reports to t...","dimondale, mi",Spring 2022
3,CRM Data Analyst,Zeiss Industrial Quality Solutions,"$60,000 - $70,000 a year",responsible for maintaining sap/crm business p...,"wixom, mi 48393+1 location",Spring 2022
4,Data Analyst,Winwire Technologies,NaN,providing resource to add/delete/amend data wi...,remote,Spring 2022


In [8]:
import re
indeed_df['salary'] = indeed_df['salary'].str.replace(',','')
indeed_df['salary'] = indeed_df['salary'].str.replace('-',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")   


C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/4190113625.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  indeed_df['salary'] = indeed_df['salary'].str.replace('$',' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/4190113625.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['salary'] = indeed_df['salary'].str.replace("[a-z]","")


In [9]:
income_split = indeed_df['salary'].str.split(expand = True)
indeed_df['salary_lower_range'] =  income_split[0]
indeed_df['salary_upper_range'] =  income_split[1]
indeed_df.drop('salary',axis = 1,inplace=True)

In [10]:

indeed_df['company'].dropna(axis=0,inplace=True)
indeed_df['salary_lower_range'].fillna(0,inplace=True)
indeed_df['salary_upper_range'].fillna(0,inplace=True)
indeed_df['salary_lower_range'] = indeed_df['salary_lower_range'].astype(float)
indeed_df['salary_upper_range']  = indeed_df['salary_upper_range'].astype(float)
hourly_salaries = indeed_df[indeed_df['salary_lower_range'] < 100]
indeed_df = indeed_df[indeed_df['salary_lower_range'] > 100]

In [11]:
hourly_salaries['salary_lower_range']  = (hourly_salaries['salary_lower_range'] * 40) * 52
hourly_salaries['salary_upper_range'] = (hourly_salaries['salary_upper_range'] * 40) * 52
indeed_df['salary_lower_range'] = indeed_df['salary_lower_range'].astype(int)
indeed_df['salary_upper_range']  = indeed_df['salary_upper_range'].astype(int)

In [12]:
indeed_df = pd.concat([hourly_salaries,indeed_df])

In [13]:
indeed_df['is_remote'] = np.where(indeed_df['description'].str.contains('remote') | indeed_df['location'].str.contains('remote') ,1,0)
indeed_df[indeed_df['is_remote'] == 1]

,title,company,description,location,datadate,salary_lower_range,salary_upper_range,is_remote
4,Data Analyst,Winwire Technologies,providing resource to add/delete/amend data wi...,remote,Spring 2022,0.0,0.0,1
5,Junior Data Analyst - DAL22-033349999999,"Evergreen Technologies, LLC.","experience with data wrangling, data cleaning ...","remote in detroit, mi+7 locations",Spring 2022,0.0,0.0,1
6,Junior Data Analyst,Teamson,at least two years of experience as an analyst...,remote,Spring 2022,0.0,0.0,1
8,"Senior Data Analyst, IT Remote",U.S. Medical Management,perform data cleansing to identify erroneous d...,"remote in troy, mi 48084",Spring 2022,0.0,0.0,1
12,Data Analyst,Spokeo,we collect data from a wide variety of public ...,"remote in pasadena, ca 91101",Spring 2022,0.0,0.0,1
...,...,...,...,...,...,...,...,...
492,Fraud Business Intelligence Data Analyst - Car...,NaN,provide concise data reports and clear data vi...,remote•remote,Fall 2021,90000.0,100000.0,1
1347,"Senior Program & Engagement Officer, Data Scie...",NaN,"deep knowledge of data science, collections me...",united states•remote,Fall 2021,80000.0,100000.0,1
1360,"Vice President, Data Science/Advanced Analytics",NaN,lead rdna's data strategy including data stora...,maryland•remote,Fall 2021,139000.0,191000.0,1
1369,"Manager, Data Science & Analytics",NaN,expert knowledge in data analysis and preparat...,remote•remote,Fall 2021,110700.0,135300.0,1


In [14]:
indeed_df['location'] = indeed_df['location'].str.replace('remote','')
indeed_df['location'] = indeed_df['location'].str.replace('in','')
indeed_df.groupby('is_remote').count()

,title,company,description,location,datadate,salary_lower_range,salary_upper_range
is_remote,,,,,,,
0,1766,1031,1758,1764,1766,1766,1766
1,1494,1313,1494,1494,1494,1494,1494


In [15]:
indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] == '',0,indeed_df['zipcode'])
indeed_df['zipcode'] = indeed_df['zipcode'].fillna(0)
indeed_df['zipcode'] = indeed_df['zipcode'].astype('int')
indeed_df['zipcode'] = np.where(indeed_df['zipcode'] < 1000,0,indeed_df['zipcode'])
indeed_df.drop_duplicates(inplace = True)

C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/730417472.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['zipcode'] = indeed_df['location'].str.replace(r'\D+','')


In [16]:
indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/1525615495.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['location'] = indeed_df['location'].str.replace("[0-9]",'')


In [17]:
indeed_df['location'] = indeed_df['location'].str.strip()
indeed_df['location'] = indeed_df['location'].str.split(',')

In [18]:
indeed_df['city'] = indeed_df['location'].str[0]
indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
indeed_df['state'] = indeed_df['location'].str[1]
indeed_df['state'] = indeed_df['state'].str.split()
indeed_df['state'] = indeed_df['state'].str[0]
indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/3846399863.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['city'] = indeed_df['city'].str.replace("[0-9+()-]",' ')
C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/3846399863.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['state'] = indeed_df['state'].str.replace("[0-9+()-]",'')


In [19]:
indeed_df['city'] = np.where(indeed_df['city'].str.contains('location'),'Remote',indeed_df['city'])
indeed_df.groupby('state').count()
indeed_df.groupby('city').count()           
indeed_df.drop('location',axis = 1,inplace=True)

In [20]:
indeed_df['title'] = indeed_df['title'].str.replace('[0-9]','')

C:\Users\Nicho\AppData\Local\Temp/ipykernel_2420/1637865698.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  indeed_df['title'] = indeed_df['title'].str.replace('[0-9]','')


In [21]:
indeed_df['title_temp'] = indeed_df['title'].str.lower()
indeed_df['job_type'] = np.where(indeed_df['title_temp'].str.contains('analy'),'Data Analyst','0')
indeed_df['job_type'] = np.where(indeed_df['title_temp'].str.contains('engineer'),'Data Engineer',indeed_df['job_type'])
indeed_df['job_type'] = np.where(indeed_df['title_temp'].str.contains('scien'),'Data Scientist',indeed_df['job_type'])
indeed_df['job_type'] = np.where(indeed_df['job_type'] == '0','Other',indeed_df['job_type'])

In [22]:
indeed_df[indeed_df['job_type'] == 'Other']

,title,company,description,datadate,salary_lower_range,salary_upper_range,is_remote,zipcode,city,state,title_temp,job_type
39,Data Research Master,ucd,"we need a researcher to help gather, organize,...",Spring 2022,45760.0,0.0,1,0,,NaN,data research master,Other
431,Data Specialist,Relation Insurance,responsible for data conversion and merger act...,Spring 2022,0.0,0.0,1,0,Remote,NaN,data specialist,Other
805,Data Reporting Specialist,Michigan Public Health Institute (MPHI),the data reporting analyst will also provide s...,Spring 2022,66580.8,87380.8,1,0,lansg,mi,data reporting specialist,Other
924,Reporting and Data Specialist,Greater Nevada Credit Union,in-depth understanding of underlying data and ...,Spring 2022,0.0,0.0,1,0,united states,NaN,reporting and data specialist,Other
966,part-time Information Systems Specialist - Bus...,Oregon Youth Authority,"strong ability to analyze and qa data, identif...",Spring 2022,70616.0,106724.8,0,973012,salem,or,part-time information systems specialist - bus...,Other
...,...,...,...,...,...,...,...,...,...,...,...,...
1264,Postdoctoral Researcher,NaN,demonstrated experience performing data analys...,Fall 2021,57000.0,59000.0,0,0,fort colls,co,postdoctoral researcher,Other
1737,Postdoctoral Research Associate IDRT,NaN,recent peer-reviewed publications in computer ...,Fall 2021,3000.0,0.0,0,77554,galveston,tx,postdoctoral research associate idrt,Other
1755,Statistician (Operations and Administration),NaN,analyze difficult statistics and other qualita...,Fall 2021,72750.0,94581.0,0,20415,washgton,dc,statistician (operations and administration),Other
1766,Interdisciplinary Senior Mathematical Statisti...,NaN,contributing to primary data collection.advisi...,Fall 2021,97481.0,136749.0,0,0,washgton,dc,interdisciplinary senior mathematical statisti...,Other


In [23]:
experience_levels = ['associate','senior','intern','staff','principal',
                  'director','leader','manager','specialist','entry level','machine learning'
                    ,'junior','jr','sr']

indeed_df['experience_level'] = '0'
for index,level in enumerate(experience_levels):
    indeed_df['experience_level'] = np.where(indeed_df['title_temp'].str.contains(level),level,indeed_df['experience_level'])
            
            

In [24]:
indeed_df['experience_level'].value_counts()

0                   2367
senior               244
machine learning     131
sr                   130
associate            103
manager               66
junior                61
specialist            34
director              31
principal             26
jr                    21
entry level           20
staff                 16
leader                 7
intern                 3
Name: experience_level, dtype: int64

In [25]:
indeed_df.drop('title_temp', axis = 1, inplace = True)

In [26]:
indeed_df.drop_duplicates(inplace=True)
indeed_df.to_csv(r'C:\Users\Nicho\gc-march-2022\data\IndeedCleanedData.csv', index=False)